<a href="https://colab.research.google.com/github/JWNLP/nlp/blob/main/21_04_09%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 2.2MB 7.9MB/s 
     |████████████████████████████████| 3.3MB 51.2MB/s 
     |████████████████████████████████| 870kB 60.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=24f99102dbb49d9fd9aa23ea21c8c8de6cce8a93cb39a97f7bfe636b6c6d0c29
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [7]:
train = pd.read_csv("/content/drive/MyDrive/케글/train.hate.csv", sep=',')
dev = pd.read_csv("/content/drive/MyDrive/케글/dev.hate.csv", sep=',')
test = pd.read_csv("/content/drive/MyDrive/케글/test.hate.no_label.csv", sep=',')

print(train.info)
print(dev.info)
print(test.info)

<bound method DataFrame.info of                                                comments label
0     (현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...  hate
1     ....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...  none
2     ...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...  hate
3                    1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데  none
4     1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...  hate
...                                                 ...   ...
7891                                      힘내세요~ 응원합니다!!  none
7892                             힘내세요~~삼가 고인의 명복을 빕니다..  none
7893                              힘내세용 ^^ 항상 응원합니닷 ^^ !  none
7894  힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...  none
7895                                 힘들면 관뒀어야지 그게 현명한거다  none

[7896 rows x 2 columns]>
<bound method DataFrame.info of                                               comments      label
0                          송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.       none
1                                 

In [8]:
train.head(10)

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate
5,10+8 진짜 이승기랑 비교된다,none
6,"100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, ...",hate
7,10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음,none
8,10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐,offensive
9,10년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름...,offensive


In [9]:
# 리뷰 문장 추출
sentences = train['comments']
sentences[:10]

0    (현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...
1    ....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...
2    ...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...
3                   1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데
4    1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...
5                                    10+8 진짜 이승기랑 비교된다
6    100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, ...
7                   10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음
8                   10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐
9    10년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름...
Name: comments, dtype: object

In [10]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] (현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네.... [SEP]',
 '[CLS] ....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을 미처 알지못했네요ㅠ [SEP]',
 '[CLS] ...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래야, 공정한 사회지...심은대로 거두거라... [SEP]',
 '[CLS] 1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데 [SEP]',
 '[CLS] 1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각이 없노 [SEP]',
 '[CLS] 10+8 진짜 이승기랑 비교된다 [SEP]',
 '[CLS] 100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, 여성 투표권 삭제가 세계의 공통문화로 자리잡을듯. 암탉이 너무 울어댐. [SEP]',
 '[CLS] 10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음 [SEP]',
 '[CLS] 10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐 [SEP]',
 '[CLS] 10년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름기억은해둠ㅇㅇ [SEP]']

In [11]:
train['label'] =train['label'].replace({"none":0,"offensive":1,"hate":2})
labels=train['label'].values
labels

array([2, 0, 2, ..., 0, 0, 0])

In [12]:
'''
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(train['label'])
train['label'] = encoder.fit_transform(train['label'])
labels=train['label'].values
#train.info()
labels
#hate:0, none:1, offensive:2
'''

"\nfrom sklearn.preprocessing import LabelEncoder\n\nencoder = LabelEncoder()\nencoder.fit(train['label'])\ntrain['label'] = encoder.fit_transform(train['label'])\nlabels=train['label'].values\n#train.info()\nlabels\n#hate:0, none:1, offensive:2\n"

In [13]:
train['label']

0       2
1       0
2       2
3       0
4       2
       ..
7891    0
7892    0
7893    0
7894    0
7895    0
Name: label, Length: 7896, dtype: int64

In [14]:
# 라벨 추출
labels = train['label'].values
labels

array([2, 0, 2, ..., 0, 0, 0])

In [15]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])


[CLS] (현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네.... [SEP]
['[CLS]', '(', '현재', '호', '##텔', '##주', '##인', '심', '##정', ')', '아', '##18', '난', '마', '##른', '##하', '##늘', '##에', '날', '##벼', '##락', '##맞', '##고', '호', '##텔', '##망', '##하게', '##생', '##겼', '##는데', '누', '##군', '계속', '추', '##모', '##받', '##네', '.', '.', '.', '.', '[SEP]']


In [16]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    113,  26565,   9985, 119354,  16323,  12030,   9491,
        16605,    114,   9519,  45987,   8984,   9246,  37819,  35506,
       118762,  10530,   8985, 118983, 107693, 118913,  11664,   9985,
       119354,  89292,  17594,  24017, 118637,  41850,   9032,  17360,
        77039,   9765,  39420, 118965,  77884,    119,    119,    119,
          119,    102,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [17]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [18]:
#train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, train['label'].values, random_state=42, test_size=0.1)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=42, 
                                                       test_size=0.1)



# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   8853,  11903,  25503,  96720, 110176,  26344,  28911,    132,
           132,    132,    102,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [19]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 전처리-테스트셋

In [20]:
# 리뷰 문장 추출
sentences = dev['comments']
sentences[:10]

0                          송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.
1                                              지현우 나쁜놈
2           알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라
3                                     설마 ㅈ 현정 작가 아니지??
4    이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...
5                                     일베충들 ㅂㄷ거리는것봐라 ㅉㅉ
6    아이즈원 힘내세요...일본 진출도 했으니 일본서 좋은 모습 보여줘도 팬들은 응원 합니다.
7                강부자 선생님 전미선 비보에 오열을 하셨다니 눈물이 나네요 힘내세요
8                                               알았어 그만
9    이영자씨는 진정성 있는거라면 녹화불참으로 끝내지말고 자진하차해라 시청자는 고려도 안...
Name: comments, dtype: object

In [21]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 송중기 시대극은 믿고본다. 첫회 신선하고 좋았다. [SEP]',
 '[CLS] 지현우 나쁜놈 [SEP]',
 '[CLS] 알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라 [SEP]',
 '[CLS] 설마 ㅈ 현정 작가 아니지?? [SEP]',
 '[CLS] 이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀 하고사시죠. [SEP]',
 '[CLS] 일베충들 ㅂㄷ거리는것봐라 ㅉㅉ [SEP]',
 '[CLS] 아이즈원 힘내세요...일본 진출도 했으니 일본서 좋은 모습 보여줘도 팬들은 응원 합니다. [SEP]',
 '[CLS] 강부자 선생님 전미선 비보에 오열을 하셨다니 눈물이 나네요 힘내세요 [SEP]',
 '[CLS] 알았어 그만 [SEP]',
 '[CLS] 이영자씨는 진정성 있는거라면 녹화불참으로 끝내지말고 자진하차해라 시청자는 고려도 안하고 일방적 불참은 아닌듯 엠비씨도 시청율 좋아서 고민하는거 같은데 결방할게 아니고 폐지해라 [SEP]']

In [22]:
dev['label'] =dev['label'].replace({"none":0,"offensive":1,"hate":2})
labels=dev['label'].values
labels

array([0, 1, 2, 2, 1, 2, 0, 0, 1, 1, 0, 2, 0, 1, 1, 1, 2, 2, 2, 1, 1, 0,
       1, 2, 2, 0, 1, 2, 1, 2, 0, 2, 0, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 1,
       0, 1, 2, 2, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 2, 0,
       1, 1, 0, 2, 1, 1, 0, 2, 1, 0, 1, 2, 1, 1, 0, 1, 2, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 2, 1, 2, 2, 1, 0, 1,
       0, 1, 2, 1, 1, 1, 0, 0, 2, 2, 1, 0, 2, 0, 1, 0, 0, 2, 2, 0, 0, 0,
       2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 2, 0, 2, 1, 2, 0, 1,
       1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 2, 2,
       1, 0, 0, 2, 0, 2, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1, 2, 1,
       1, 1, 0, 2, 0, 2, 1, 0, 0, 2, 0, 1, 2, 2, 2, 2, 0, 2, 1, 1, 2, 2,
       1, 0, 0, 2, 2, 2, 1, 0, 0, 1, 1, 2, 1, 0, 0, 1, 0, 1, 1, 1, 0, 2,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 2, 0, 2, 1, 0, 0, 1, 1, 0, 0, 1, 2,
       2, 0, 0, 1, 1, 2, 1, 2, 1, 2, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 2, 1,
       2, 0, 0, 2, 0, 2, 2, 1, 0, 1, 0, 0, 2, 1, 0,

In [23]:
# 라벨 추출
labels = dev['label'].values
labels

array([0, 1, 2, 2, 1, 2, 0, 0, 1, 1, 0, 2, 0, 1, 1, 1, 2, 2, 2, 1, 1, 0,
       1, 2, 2, 0, 1, 2, 1, 2, 0, 2, 0, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 1,
       0, 1, 2, 2, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 2, 0,
       1, 1, 0, 2, 1, 1, 0, 2, 1, 0, 1, 2, 1, 1, 0, 1, 2, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 2, 1, 2, 2, 1, 0, 1,
       0, 1, 2, 1, 1, 1, 0, 0, 2, 2, 1, 0, 2, 0, 1, 0, 0, 2, 2, 0, 0, 0,
       2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 2, 0, 2, 1, 2, 0, 1,
       1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 2, 2,
       1, 0, 0, 2, 0, 2, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1, 2, 1,
       1, 1, 0, 2, 0, 2, 1, 0, 0, 2, 0, 1, 2, 2, 2, 2, 0, 2, 1, 1, 2, 2,
       1, 0, 0, 2, 2, 2, 1, 0, 0, 1, 1, 2, 1, 0, 0, 1, 0, 1, 1, 1, 0, 2,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 2, 0, 2, 1, 0, 0, 1, 1, 0, 0, 1, 2,
       2, 0, 0, 1, 1, 2, 1, 2, 1, 2, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 2, 1,
       2, 0, 0, 2, 0, 2, 2, 1, 0, 1, 0, 0, 2, 1, 0,

In [24]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 송중기 시대극은 믿고본다. 첫회 신선하고 좋았다. [SEP]
['[CLS]', '송', '##중', '##기', '시대', '##극', '##은', '믿', '##고', '##본', '##다', '.', '첫', '##회', '신', '##선', '##하고', '좋', '##았다', '.', '[SEP]']


In [25]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9454,  41693,  12310, 102080,  63243,  10892,   9312,
        11664,  40419,  11903,    119,   9750,  14863,   9487,  18471,
        12453,   9685,  27303,    119,    102,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [26]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [27]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9454,  41693,  12310, 102080,  63243,  10892,   9312,  11664,
         40419,  11903,    119,   9750,  14863,   9487,  18471,  12453,   9685,
         27303,    119,    102,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [28]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

#모델생성

In [29]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [30]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [31]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [32]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 모델학습

In [33]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [34]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [35]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]
        
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 1.01
  Training epcoh took: 0:02:33

Running Validation...
  Accuracy: 0.55
  Validation took: 0:00:06

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.85
  Training epcoh took: 0:02:37

Running Validation...
  Accuracy: 0.57
  Validation took: 0:00:06

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.74
  Training epcoh took: 0:02:37

Running Validation...
  Accuracy: 0.58
  Validation took: 0:00:06

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.65
  Training epcoh took: 0:02:37

Running Validation...
  Accuracy: 0.59
  Validation took: 0:00:06

Training complete!


# 테스트 셋 평가

In [36]:

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
 

    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.59
Test took: 0:00:04


# 새로운 문장 테스트

In [37]:
# 리뷰 문장 추출
test_sentences = test['comments']
test_sentences[:10]

0         ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ
1                                        둘다 넘 좋다~행복하세요
2                 근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
3                원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요
4                                   장현승 얘도 참 이젠 짠하다...
5    신선하게 웃긴다ㅋㅋㅋ역시 동엽신~~!! 장소연님은 진짜 조선족인가 착각할정도로 말투...
6                                              누군데 얘네?
7    하자 인생들 모아다가 방송에 내보내고, 덜 하자가 교정해서 장사 풀리게 해주는 감동...
8                       진짜 라디오 스타 노래한거 보세요 홍진영은비비지도 못함
9    쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ...
Name: comments, dtype: object

In [38]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [39]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [40]:
logits = test_sentences(['쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금'])

print(logits)
print(np.argmax(logits))

[[-1.6806663   0.45488653  1.5701103 ]]
2


In [41]:
logits = test_sentences(test['comments'])
print(logits)
print(np.argmax(logits))

[[-0.70761395  0.48789537  0.4285394 ]
 [ 2.7138507  -1.2458563  -2.3079293 ]
 [ 0.5947649   0.43599132 -1.2689676 ]
 ...
 [ 1.1982716   0.03250477 -2.0190582 ]
 [-0.03112631  0.7296083  -0.56435144]
 [ 1.9408195  -0.3706167  -2.3557801 ]]
105


In [45]:
logits = test_sentences(test['comments']) 
    #logits = test_sentences(['누군지도모르는데 펜이있어?음......참....재인씨 고생고생해서 자리잡은사람 가기고놀지마세요천벌받는다'])

    #print(test_results)
test_labels = [np.argmax(i) for i in logits]
    #test_labels = [le.inverse_transform(i) for i in test_results] 
no_label = pd.DataFrame({'comments':test['comments'],"label":test_labels})
no_label.to_csv("/content/NEW_test.hate.no_label.csv",index=False)



In [47]:
no_label

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,1
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,1
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,2
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
972,입에 손가릭이 10개 있으니 징그럽다,1
